<a href="https://colab.research.google.com/github/David-Carballo/2023w-project-pose/blob/main/features_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MobileNet v2

##0. Load DataSet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"


In [ ]:
! mkdir data

In [ ]:
#! rm -rf data

In [ ]:
#! unzip 'drive/MyDrive/dataset.zip' -d data/
! unzip 'drive/MyDrive/minim_dataset.zip' -d data/

##1. Model implementation

In [ ]:
import torch
import os
from PIL import Image
from torchvision import transforms

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.eval()

In [ ]:
# sample execution (requires torchvision)
dataset_path = '/content/data/minim_dataset/black'

# directorios de carpetas
train_dir = os.path.join(dataset_path,'train')
val_dir = os.path.join(dataset_path,'validation')
test_dir = os.path.join(dataset_path,'test')
directories = [train_dir, val_dir, test_dir]

for dir in directories:
    dir_path = os.path.join(dataset_path, dir)
    for classname in os.listdir(dir_path):
        class_path = os.path.join(dir_path, classname)
        for filename in os.listdir(class_path):
            file_path = os.path.join(class_path, filename)
            input_image = Image.open(file_path)
            preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

            # move the input and model to GPU for speed if available
            if torch.cuda.is_available():
                input_batch = input_batch.to('cuda')
                model.to('cuda')

            with torch.no_grad():
                output = model(input_batch)
            # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
            print(output[0])
            # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
            probabilities = torch.nn.functional.softmax(output[0], dim=0)
            print(probabilities)

In [ ]:
# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

In [ ]:
with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)